<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/Remote_sensing_2021/blob/main/R_code_classification.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_classification.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Image Classification
The classification of remotely sensed images consists in associating each pixel of the image with a specific class or category (e.g. grassland, forest, water, soil, etc.).
Identification occurs through a process in which the degree of spectral similarity or association is high between pixels of the same category and low between pixels of distinct categories.
> For this first application of the classification methods we use two sets of images: 
* the recent images of [Solar Orbiter](http://www.esa.int/Science_Exploration/Space_Science/Solar_Orbiter) with different energy classes of the solar surface;
* and the Landsat images of the [Grand Canyon](https://landsat.visibleearth.nasa.gov/view.php?id=80948) with different morphologies and types of rocks.

In [6]:
solar_orbiter_url <- "https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2020/07/solar_orbiter_s_first_views_of_the_sun4/22133246-1-eng-GB/Solar_Orbiter_s_first_views_of_the_Sun.jpg"
grand_canyon_url <- "https://eoimages.gsfc.nasa.gov/images/imagerecords/80000/80948/dolansprings_oli_2013088_canyon_lrg.jpg"

The [download.file](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/download.file) function can be used to download a file from the Internet. 
```
download.file(url, destfile)
```
In the function the `url` argument sets the resource to be downloaded and the `destfile` a character string with which the downloaded file is named.

In [7]:
download.file(solar_orbiter_url, "Solar_Orbiter.jpg")
download.file(grand_canyon_url, "Grand_Canyon.jpg")

### Install R packages
The [install.packages](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/install.packages) function is used to download and install packages from CRAN-like repositories.
To install multiple packages at the same time, you define a vector that contains the names of the packages to be installed.
The [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-10) package defines classes and methods for spatial raster data access and manipulation. 
[RStoolbox](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6) is an package providing a wide range of tools for your every-day remote sensing processing needs. The available tool-set covers many aspects for remote sensing image processing and analysis such as calculating spectral indices, principal component transformation, unsupervised and supervised classification or fractional cover analyses.

In [ ]:
packages <- c("raster", "RStoolbox")
install.packages(packages)

The [library](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/library) function is used to load previously installed packages.
Using the [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) function we simultaneously apply the `library()` function to all elements of the vector containing the packages.

In [ ]:
lapply(packages , library, character.only = TRUE)

## Solar Orbiter
During the seven-year mission, instruments aboard the spacecraft will measure solar wind plasma, electromagnetic fields, and energetic particles in an area close enough to the sun. Remote sensing tools, on the other hand, will send spectacular images of solar characteristics with a resolution never achieved before and provide new information on what determines the cyclicality of sunspots and the onset of flares.

### Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function (RasterBrick object). 
Explore the properties of the `solar_orbiter` object.

In [ ]:
solar_orbiter <- raster::brick("Solar_Orbiter.jpg")
solar_orbiter

### Visualization of spatial data
Several generic functions have been implemented for "raster" objects to create maps and other plot types. For example, use [plot](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plot) function to create a map of a raster object.
In addition, individual bands can be combined into a single image by assigning individual layers or bands to one of the three color channels (red, green and blue). 
The [plotRGB](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plotRGB) function can be used to make "true (or false) color images" from multi-band satellite images.
In this case, the resulting color tone in the image will be linked to the combination of the different energy values of the solar surface.


In [ ]:
plotRGB(solar_orbiter, 1,2,3, stretch="lin")

### Unsupervised clustering
[K-means](https://en.wikipedia.org/wiki/K-means_clustering) clustering is one of the simplest and popular unsupervised learning algorithms.
Typically, unsupervised algorithms make inferences from datasets using only input vectors without referring to known, or labeled, outcomes.

The `RStoolbox` package provides the [unsuperClass](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6/topics/unsuperClass) function which allows you to apply the k-means algorithm to any Raster Object.

Furthermore, by setting the same seed with the [set.seed](https://www.rdocumentation.org/packages/simEd/versions/2.0.0/topics/set.seed) function (a random number generator) we always select the same initial pixels during the construction phase of the n-clusters in order to always show the same output.


In [11]:
set.seed(1234)
solar_orbiter_class6 <- RStoolbox::unsuperClass(solar_orbiter, nClasses=6)

### Colour change
Use function [colorRampPalette](https://www.rdocumentation.org/packages/dichromat/versions/1.1/topics/colorRampPalette) to create a continuous color scale divided into n-levels or classes.

In [14]:
cl <- colorRampPalette(c('yellow', 'red', 'black'))(6)

### Export images
The [pdf](https://www.rdocumentation.org/packages/grDevices/versions/3.6.2/topics/pdf) function starts the graphics device driver for producing PDF graphics. 
The pdf file will be named in the path (function argument) and saved on Google Colab. Click the folder icon to view all files temporarily uploaded to the machine (click the pdf name in the list to download and view the image). 
Now you can zoom in and explore each subplot better!

N.B. always use the [dev.off](https://www.rdocumentation.org/packages/imguR/versions/0.1.5/topics/dev.off) function to manage the correct closing and loading of the graphics space.

In [ ]:
pdf(file = '/content/solar_orbiter_class6.pdf')
plot(solar_orbiter_class6$map, col=cl, axes= FALSE)
dev.off()

## Grand Canyon
"*The wonders of the Grand Canyon cannot be adequately represented in symbols or speech*", John Wesley Powel wrote in his log. "*The resources of the graphic art are taxed beyond their powers in attempting to portray its features"*.

Powell was, of course, seeing the canyon mainly from river level; there was no technology that provided views of the landscape from space then. If there had been, he would have seen something similar to what the Operational Land Imager (OLI) on the [Landsat Data Continuity Mission](https://landsat.gsfc.nasa.gov/landsat-8) (LDCM) did on March 29, 2013. 

### Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function (RasterBrick object). 
Explore the properties of the `grand_canyon` object.

In [ ]:
grand_canyon <- brick("Grand_Canyon.jpg")
grand_canyon

### Visualization of spatial data
In this case, the resulting color differences in the image will be related to lithological differences, the presence or absence of water, the almost absent vegetative cover and the presence of clouds.



In [ ]:
plotRGB(grand_canyon, r=1, g=2, b=3, stretch="lin")

In [ ]:
plotRGB(grand_canyon, r=1, g=2, b=3, stretch="hist")

### Unsupervised clustering
Typically, unsupervised algorithms make inferences from datasets using only input vectors without referring to known or labeled results.
Therefore the content of each class should be validated in the field or compared with other data demonstrating belonging to the chosen land cover type. This step is very important and is useful for defining the accuracy of the classification model used.




In [26]:
set.seed(1234)
grand_canyon_class6 <- unsuperClass(grand_canyon, nClasses=6)
cl2 <- colorRampPalette(c('purple', 'blue', 'yellow', 'red', 'black'))(6)

### Export images
The [pdf](https://www.rdocumentation.org/packages/grDevices/versions/3.6.2/topics/pdf) function starts the graphics device driver for producing PDF graphics. 
The pdf file will be named in the path (function argument) and saved on Google Colab. Click the folder icon to view all files temporarily uploaded to the machine (click the pdf name in the list to download and view the image). 
Now you can zoom in and explore each subplot better!

N.B. always use the [dev.off](https://www.rdocumentation.org/packages/imguR/versions/0.1.5/topics/dev.off) function to manage the correct closing and loading of the graphics space.

In [34]:
pdf(file = '/content/grand_canyon_class6.pdf')
plot(grand_canyon_class6$map, col=cl2, axes= FALSE)
dev.off()